In [ ]:
from langdetect import detect
from langdetect import detect_langs
from langdetect import DetectorFactory
import nltk
import string
from nltk.stem import WordNetLemmatizer
import numpy as np
import re

def preprocess(text):
    
    lemmatizer = WordNetLemmatizer()
    DetectorFactory.seed = 0
    punctuation = "ξ"
    tokens = [token.strip(punctuation) for token in nltk.word_tokenize(text)]
    text = " ".join(tokens)
#     text = re.sub('\W+'," ", text)
    if len(text) >= 200:
#         langs = detect_langs(text)
#         main_lang = detect(text)
        return text
    return np.nan

In [ ]:
import itertools
import string
import nltk
def extract_candidate_words(
        text,
        good_tags=set(['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'NNPS', 'VB', 'VBD',
                      'VBG', 'VBN', 'VBP', 'VBZ'])
        ):
    '''Exclude candidates that are stop words or entirely punctuation.
    '''
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize and POS-tag words
    tagged_words = itertools.chain.from_iterable(
        nltk.pos_tag_sents(
            nltk.word_tokenize(sent) 
            for sent in nltk.sent_tokenize(text)
        )
    )
    # filter on certain POS tags and lowercase all words
    candidates = [word.lower() for word, tag in tagged_words
                  if tag in good_tags and word.lower() not in stop_words
                  and not all(char in punct for char in word)]

    return candidates

In [ ]:
import trie_search
def create_trie_from_patterns(file_path):
    file = open(file_path, "r", encoding="utf-8")
    patterns = ''.join(file.readlines())
    patterns = patterns.splitlines()
    file.close()
    return trie_search.TrieSearch(patterns)

In [ ]:
def annotate_words(words, tag, candidates):
    res = ""
    chunk = "B-"
    for word in nltk.word_tokenize(words):
        if word.lower() not in candidates:
            tag_curr = 'O'
            chunk = "B-"
        elif tag != "O":
            tag_curr = chunk + tag
            chunk = "I-"
        else:
            tag_curr = tag
        res += word + " " + tag_curr + "\n"
    return res

In [ ]:
from nltk.corpus import stopwords
def annotate_text(text):
    text = "\n" + text + "\n"
    trie = create_trie_from_patterns("data/skills_lower.txt")
    annotated_text = ""
    candidates = extract_candidate_words(text)
    for sentence in nltk.sent_tokenize(text):
        tokens = nltk.word_tokenize(sentence)
        sentence = " ".join(tokens)
        end = 0
        for pattern, start in sorted(trie.search_longest_patterns(sentence.lower()),
                                     key=lambda x: x[1]):
            if start != end:
                words = sentence[end:start].strip()
                res = annotate_words(words, "O", candidates)
                annotated_text += res
            end = start + len(pattern)
            
            res = annotate_words(sentence[start:end], "SKL", candidates)
            annotated_text += res
            
        words = sentence[end:].strip()
        res = annotate_words(words, "O", candidates)
        annotated_text += res + "\n"
    return annotated_text

In [ ]:
file = open("data/sample_CV.txt", 'r', encoding="utf-8")
text = "".join(file.readlines())
file.close()
words_tag = annotate_text(text)
print(words_tag)

# Annotation of data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/wcn_cv.csv', index_col=0)
df.head()

In [ ]:
from multiprocessing import Pool
import time
start_time = time.time()
texts = list(df['cv_txt'])
pool = Pool()
annotated_texts = pool.map(annotate_text, texts[:2000])
pool.close()
pool.join()
    
end_time = time.time()
print(end_time - start_time)

In [ ]:
import math
file = open("sequence_tagging/data/train.txt", "w", encoding="utf-8")
begin = 0
end = math.ceil(len(annotated_texts) * 0.6)
[file.writelines(text) for text in annotated_texts[begin:end]]
file.close()

In [ ]:
file = open("sequence_tagging/data/val.txt", "w", encoding="utf-8")
begin = math.ceil(len(annotated_texts) * 0.6)
end = math.ceil(len(annotated_texts) * 0.8)
[file.writelines(text) for text in annotated_texts[begin:end]]
file.close()

In [ ]:
file = open("sequence_tagging/data/test.txt", "w", encoding="utf-8")
begin = math.ceil(len(annotated_texts) * 0.8)
end = math.ceil(len(annotated_texts))
[file.writelines(text) for text in annotated_texts[begin:end]]
file.close()

In [ ]:
!wc -l sequence_tagging/data/train.txt

In [ ]:
!head sequence_tagging/data/train.txt